# CS5830 - Group15 - Project 6

## Baseflow Dataset Analysis + Linear Regression

#### Dataset

- *Date* – number of days since 01/01/0000
- *Segment id* – an identifier of the segment of river; it can be treated as a categorical variable
- *x/y* – the spatial location of the gaging station at which observations are obtained
- *Evapotranspiration* – the evapotranspiration amount of an area adjacent to the river segment in the given month
- *Precipitation* - the precipitation amount of an area adjacent to the river segment in the given month
- *Irrigation pumping* - the amount of groundwater pumped out for irrigation in an area adjacent to the river segment in the given month
- *Observed* – observed baseflow \[target\]

#### Imports

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

#### Data Preparation

In [ ]:
df = pd.read_csv("./data/RRCA_baseflow.csv")
df.info()